In [1]:
from langchain_ollama import ChatOllama
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import time

# 모델 일관성을 위한 상수 정의
MODEL_NAME = "qwen2.5:1.5b"
BASE_URL = "http://localhost:11434"
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

In [15]:
print("=" * 60)
print(" 로컬 RAG 시스템 초기화")
print("=" * 60)

# 1. 문서 로딩
print("==> 1. 문서 로딩 → PDF 읽기...")
try:
    loader = PyPDFLoader('../data/tutorial-korean.pdf')
    documents = loader.load()
    print(f"총 {len(documents)}페이지 로드 완료")
except Exception as e:
    print(f"PDF 로딩 실패: {e}")
    print("'../data/tutorial-korean.pdf' 파일이 존재하는지 확인해주세요.")
    exit(1)


 로컬 RAG 시스템 초기화
==> 1. 문서 로딩 → PDF 읽기...
총 39페이지 로드 완료


In [16]:

# 2. 문서 분할
print("==> 2. 문서 분할 → 작은 청크로 나누기")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
    separators=["\n\n", "\n", ".", " ", ""]
)

chunks = text_splitter.split_documents(documents)
print(f"{len(chunks)}개 청크 생성 완료")
print(f"평균 청크 길이: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.0f}자")

# 3. 임베딩 설정
print("==> 3. 임베딩 설정...")
print("임베딩 모델 다운로드 중... (최초 실행 시 시간이 소요됩니다)")
try:
    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )
    print("HuggingFace 임베딩 모델 설정 완료")
except Exception as e:
    print(f"임베딩 모델 설정 실패: {e}")
    print("인터넷 연결을 확인하거나 다음 패키지를 설치해주세요:")
    print("pip install sentence-transformers")
    exit(1)

==> 2. 문서 분할 → 작은 청크로 나누기
76개 청크 생성 완료
평균 청크 길이: 605자
==> 3. 임베딩 설정...
임베딩 모델 다운로드 중... (최초 실행 시 시간이 소요됩니다)
HuggingFace 임베딩 모델 설정 완료


In [17]:

# 4. 벡터스토어 생성
print("==> 4. 벡터스토어 생성...")
print("임베딩 생성 중... (시간이 소요됩니다)")

start_time = time.time()
try:
    vectorstore = FAISS.from_documents(chunks, embeddings)
    embedding_time = time.time() - start_time
    print(f"FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")
    print(f"임베딩 소요 시간: {embedding_time:.1f}초")
except Exception as e:
    print(f"벡터스토어 생성 실패: {e}")
    print("모델이 설치되어 있는지 확인해주세요: ollama pull qwen2.5:latest")
    exit(1)


==> 4. 벡터스토어 생성...
임베딩 생성 중... (시간이 소요됩니다)
FAISS 벡터스토어 생성 완료 (76개 벡터)
임베딩 소요 시간: 2.4초


In [18]:

# 5. 검색기 설정
print("==> 5. 검색기 설정...")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)
print("검색기 설정 완료")

# 6. LLM 설정
print("==> 6. LLM 설정...")
try:
    llm = ChatOllama(
        model=MODEL_NAME,
        temperature=0.1,
        num_predict=800
    )
    print("LLM 설정 완료")
except Exception as e:
    print(f"LLM 설정 실패: {e}")
    exit(1)


==> 5. 검색기 설정...
검색기 설정 완료
==> 6. LLM 설정...
LLM 설정 완료


In [19]:

# 7. 프롬프트 설정
print("==> 7. 프롬프트 설정...")
prompt_template = """당신은 BlueJ 프로그래밍 환경 전문가입니다.
아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.

문서 내용:
{context}

질문: {question}

답변 규칙:
1. 문서 내용만을 근거로 답변하세요
2. 단계별로 설명하세요  
3. 구체적인 메뉴명, 버튼명을 포함하세요
4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요

답변:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
print("프롬프트 설정 완료")


==> 7. 프롬프트 설정...
프롬프트 설정 완료


In [21]:

# 8. QA 체인 생성
print("==> 8. QA 체인 생성...")
try:
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )
    print("RAG 파이프라인 구축 완료")
except Exception as e:
    print(f"QA 체인 생성 실패: {e}")
    exit(1)


==> 8. QA 체인 생성...
RAG 파이프라인 구축 완료


In [ ]:

# 9. 테스트 실행
print("\n" + "=" * 60)
print(" 로컬 RAG 시스템 테스트")
print("=" * 60)

test_questions = [
    "BlueJ에서 객체를 생성하는 방법은 무엇인가요?",
    "컴파일 오류가 발생했을 때 어떻게 확인할 수 있나요?"
]

total_start_time = time.time()

for i, question in enumerate(test_questions, 1):
    print(f"\n【테스트 {i}/{len(test_questions)}】")
    print(f"질문: {question}")
    print("답변 생성 중...")
    
    question_start_time = time.time()
    
    try:
        result = qa_chain.invoke({"query": question})
        answer = result["result"]
        source_docs = result["source_documents"]
        
        question_time = time.time() - question_start_time
        
        print(f"\n답변: (응답시간: {question_time:.1f}초)")
        print("-" * 50)
        print(answer)
        
        print(f"\n참조 문서:")
        for j, doc in enumerate(source_docs[:2], 1):
            page = doc.metadata.get('page', 'N/A')
            preview = doc.page_content[:50].replace('\n', ' ')
            print(f"   {j}. 페이지 {page}: {preview}...")
            
    except Exception as e:
        print(f"답변 생성 실패: {e}")
    
    print("\n" + "-" * 40)

total_time = time.time() - total_start_time
print(f"\n테스트 완료! (총 소요시간: {total_time:.1f}초)")

# 10. 대화형 모드
print("\n" + "=" * 60)
print(" 대화형 모드 (종료: 'quit' 입력)")
print("=" * 60)

while True:
    try:
        user_question = input("\n질문을 입력하세요: ").strip()
        
        if user_question.lower() in ['quit', 'exit', '종료', 'q']:
            print("RAG 시스템을 종료합니다.")
            break
            
        if not user_question:
            continue
            
        print("답변 생성 중...")
        start_time = time.time()
        
        result = qa_chain.invoke({"query": user_question})
        answer = result["result"]
        response_time = time.time() - start_time
        
        print(f"\n답변: (응답시간: {response_time:.1f}초)")
        print("-" * 50)
        print(answer)
        
    except KeyboardInterrupt:
        print("\nRAG 시스템을 종료합니다.")
        break
    except Exception as e:
        print(f"오류 발생: {e}")

print("\n로컬 RAG 시스템 세션 종료!")


 로컬 RAG 시스템 테스트

【테스트 1/2】
질문: BlueJ에서 객체를 생성하는 방법은 무엇인가요?
답변 생성 중...

답변: (응답시간: 26.3초)
--------------------------------------------------
BlueJ에서 객체를 생성하는 방법은 다음과 같습니다:

1. **메뉴 선택**: 
   - `File` 메뉴에서 `New...`를 선택합니다.
   - 다음으로 `Class`를 선택하고, `Create New Class...`를 클릭합니다.

2. **클래스 이름 설정**:
   - 클래스의 이름을 입력하십시오 (예: `Person`, `Student`, 등).

3. **메서드 추가**:
   - 메서드를 작성하십시오 (예: `getName()`, `setAge(int age)` 등).
   - 이 메서드는 필요한 데이터를 저장하거나 반환합니다.

4. **클래스 파일 생성**:
   - 클래스 파일을 만듭니다.
   - 예: `Person.java` 또는 `Student.java`.

5. **클래스 파일 추가**:
   - `Project` 메뉴에서 `Add Class...`를 선택합니다.
   - 클라이언트 프로젝트 디렉토리에 클래스 파일을 선택하고, `Open in BlueJ` 버튼을 클릭합니다.

6. **클래스 파일 오픈 및 수정**:
   - 클라이언트 프로젝트에서 클래스 파일을 열고 수정하십시오.
   - 예: `Person.java`를 열어서 필요한 내용을 추가하거나 변경합니다.

7. **클래스 파일 저장**:
   - 수정된 클래스 파일을 저장하고, `Project` 메뉴에서 `Save Project...`를 선택합니다.

8. **실행 및 객체 생성**:
   - 프로젝트를 실행하십시오 (예: `$ bluej example/people`).
   - 실행 결과는 클라이언트 프로젝트 디렉토리에 나타납니다.
   - 예: `example/people/Person.java`에서 `new Per